<a href="https://colab.research.google.com/github/adrien-simard/adrien-simard-football-dataset/blob/master/EnglishPlayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English Player Dataset
The purpose of this code is to create a dataset of almost all professional football players in the English leagues.
We will catch the data from websites:

*   https://sofifa.com
*   https://fbref.com

we are going to use the library Beautifulsoup in order to collect informations on the website.

In [0]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
from bs4.element import Tag 

###Find the stats of the players
We define our function, the goal of this function is to create a dataframe with some stats about the players.
The shape of the html code is the same for each page, there are tables with the stats for each player in the league.






In [0]:
def getPlayers(Html,div):
  html = urlopen(Html)
  html_soup = BeautifulSoup(html, 'html.parser')
  rows = html_soup.findAll("div",id="all_stats_standard")
  comments =rows[0].findAll(string=lambda text: isinstance(text, Comment))
  for c in comments:
    c.extract()

  sp = BeautifulSoup(c, 'html.parser')
  l = sp.table.tbody.findAll("tr")
  players = []
  for row in range(len(l)):
    if l[row].a!=None :
      player4 = {
                  "name": l[row].a.text,
                  "Nation": l[row].findAll("td",class_="left")[1].text,
                  "Pos": l[row].findAll("td",class_="center")[0].text,
                  "Squad": l[row].findAll("td",class_="left")[2].text,
                 "division": div,
                  "Age": l[row].findAll("td",class_="center")[1].text,
                  "Born": l[row].findAll("td",class_="center")[2].text,
               "MP" : l[row].findAll("td",class_="right")[0].text,
               "Starts" : l[row].findAll("td",class_="right")[1].text,
               "Min" : l[row].findAll("td",class_="right")[2].text,
               "Gls" : l[row].findAll("td",class_="right")[3].text,
               "Ast" : l[row].findAll("td",class_="right")[4].text,
               "PK" : l[row].findAll("td",class_="right")[5].text,
               "PKatt" : l[row].findAll("td",class_="right")[6].text,
               "CrdY" : l[row].findAll("td",class_="right")[7].text,
               "CrdR" : l[row].findAll("td",class_="right")[8].text,
               "Gls/90" : l[row].findAll("td",class_="right")[9].text,
               "Ast/90" : l[row].findAll("td",class_="right")[10].text,
               "G+A" : l[row].findAll("td",class_="right")[11].text,
               "G-PK" : l[row].findAll("td",class_="right")[12].text,
               "G+A-PK" : l[row].findAll("td",class_="right")[13].text,
               "Value": None ,
              }
      players.append(player4)
  return players


We use the function for the different leagues

In [0]:
#england
df1 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/9/stats/Premier-League-Stats",1))
df2 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/10/stats/Championship-Stats",2))
df3 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/15/stats/League-One-Stats",3))
df4 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/16/stats/League-Two-Stats",4))
df5 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/34/stats/National-League-Stats",5))
tot = pd.concat([df1,df2,df3,df4,df5])
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")


In [0]:
tot = pd.concat([df1,df2,df3,df4,df5])

Read the csv file 

In [0]:
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")

In [0]:
df = pd.read_csv('ENplayers.csv')

Now we can do some requests and analysis with this dataframe.


In [0]:
R1=df.loc[ (df['Age']<=20) & (df['Gls']>=10)]
R2 = df.sort_values(by=["Gls"]).tail(30)
R3 = df.loc[(df['Age']<21)].tail(10)
R4 = df.loc[(df['Pos']== 'MF')&(df['Ast']>=7) & (df['Gls']>=4)].sort_values(by=["G+A"])

### Find the market value of the players (en cours...)
We are going to use the web site:   https://sofifa.com

In [0]:
from urllib.request import urlopen, Request
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
reg_url = "https://sofifa.com/players?type=all&lg%5B%5D=13"
req = Request(url=reg_url, headers=headers) 
html2 = urlopen(req)
html_soup = BeautifulSoup(html2, 'html.parser')

In [0]:
rows = html_soup.findAll("div",class_='column col-auto')
rows[0].findAll('td',class_="col-name")[0].text
trs =rows[0].table.findAll('tr')
trs[1].findAll('td',class_='col-name')[0].a.text
#trs[1].findAll('td',class_="col col-vl")[0].text

' V. van Dijk'

In [0]:
liste=[]
for i in range(1,len(trs)):
  player4 = {
              "name": trs[i].a.text,
              "Value": trs[i].findAll('td',class_="col col-vl")[0].text,
              }
  liste.append(player4)
liste




In [0]:
dt = pd.DataFrame(liste)
tot.loc[(tot['name'].str.contains('Gomez'))]


,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Value
162,Joe Gomez,eng ENG,DF,Liverpool,1,22,1997,19,14,"1,276",0,0,0,0,3,0,0.00,0.00,0.00,0.00,0.00,None


In [0]:
pd.merge(tot,dt,left_on= "name", right_on= "name")